In [1]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader # permet de télécharger un dataset HuggingFace
from langchain_community.embeddings import HuggingFaceEmbeddings # on récupère un embedding adapté aux données HuggingFace
from langchain.indexes.vectorstore import VectorstoreIndexCreator # permet à Langchain de transformer les données en vecteurs

In [2]:
dataset_name = "tweet_eval"
page_content_column = "text"
name = "stance_climate"

loader = HuggingFaceDatasetLoader(dataset_name, page_content_column, name)

In [5]:
embeddings = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

C:\Users\Nath\AppData\Local\Temp\ipykernel_26032\3645008222.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\Nath\AppData\Local\Temp\ipykernel_26032\3645008222.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
c:\Users\Nath\Desktop\RAG_CLIMATE_1.0\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readt

In [6]:
print(type(index))

<class 'langchain.indexes.vectorstore.VectorStoreIndexWrapper'>


In [11]:
from dotenv import load_dotenv 
import os

load_dotenv() # output = True : le code permet de récupérer les informations directement dans le fichier .env, True signifiant que le code fonctionne correctement 

True

In [13]:
api_key = ["HUGGINGFACEHUB_API_TOKEN"] # os.environ permet de récupérer les variables du fichier .env, ici on récupère la clé api(de cette manière la clé api est sécurisée dans un fichier .env à part)
# print(api_key) ne pas afficher la clé api car le git push ne fonctionnerait pas

In [17]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2", # on spécifie le modèle ici
    task="text-generation",
    device=-1, # valeur 0 fait les calculs GPU, valeur -1 fait les calculs CPU
    pipeline_kwargs={"max_new_tokens": 50}, # génère un certain nbre de nouveaux mots, nouveaux tokens (c'est la taille de la prédiction réalisée)
    model_kwargs={"temperature": 0} # permet de régler à quel point la sortie du modèle est aléatoire
)

c:\Users\Nath\Desktop\RAG_CLIMATE_1.0\rag_env\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\Nath\Desktop\RAG_CLIMATE_1.0\rag_env\Lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cpu


In [20]:
def generate_answer(query):
    result = index.query(query, llm=hf)
    target_string = "Helpful Answer:"
    return result[result.find(target_string)+len(target_string):]

In [19]:
query = "Generate a tweet about climate change"

generate_answer(query)

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n"What do you do to help reverse climate change? Tweet your tips to inspire others! #Tip #SavePolarBear #SemST"\n\n"#Follow @user US leader in weather #riskmanagement . Helping the world\'s #economy adapt to climate change. #SemST"\n\n"We\'re growing vegetables at home. Tell your friends and followers what you\'re doing for climate! #myChangeForClimate #SemST"\n\n"#CSOTA is trending at #3 in Canada on Twitter Now w/ 3,800 Tweets from the #Climate Summit of the Americas! #sm #socialmedia #SemST"\n\nQuestion: Generate a tweet about climate change\nHelpful Answer:\n\nThis question answers a question about how to create a #ClimateTweet. To find those words you add a note to your Twitter profile. Please note that this is a different Tweet format than Tweet in which you use the word climate here. This'

In [21]:
query = "What is climate change"

generate_answer(query)

' Climate Change is a global event. In the context of science, it might sound reasonable to conclude that the world will warm during the next half-decade. Yet that "global warming" theory is not credible at all. Climate change is the result'